# Examen ETL: SPARK 09/02

Se podrá utilizar toda la información que se encuentra en el campus. 

El fichero de datos sobre el que se trabajará es el de DataSetPartidos.txt.

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen o en su defecto por email.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

## Primera pregunta: ¿ En que se diferencian las acciones de las transformaciones en SPARK? ¿Podrías ponerme un par de ejemplo de cada una de ellas y describir que hacen, para que sirve?

Las transformaciones son lazy, devuelven un RDD. Las acciones ejecutan la composición de funciones lazy y devuelven un spark data frame o una lista. Transformaciones: map, filter, group.by, reducebykey. Acciones: Take, collect, Takeordered,count, sum, mean.

## Segunda tarea: Inicializar spark context y cargar los datos desde el fichero.

In [1]:
from pyspark import SparkContext

sc = SparkContext()


In [5]:
data_file = "./DataSetPartidos.txt"
raw_data = sc.textFile(data_file)


In [6]:
raw_data.take(5)

['1::1970-71::1::Athletic Club::Barcelona::1::1::12/09/1970::21942000.0',
 '2::1970-71::1::Las Palmas::Atletico de Madrid::1::1::12/09/1970::21942000.0',
 '3::1970-71::1::Real Madrid::Valencia::2::0::12/09/1970::21942000.0',
 '4::1970-71::1::Celta de Vigo::Sporting de Gijon::2::0::13/09/1970::22028400.0',
 '5::1970-71::1::Elche::Granada::1::1::13/09/1970::22028400.0']

In [8]:
parse_data = raw_data.map(lambda linea: linea.split("::"))

In [9]:
parse_data.take(2)

[['1',
  '1970-71',
  '1',
  'Athletic Club',
  'Barcelona',
  '1',
  '1',
  '12/09/1970',
  '21942000.0'],
 ['2',
  '1970-71',
  '1',
  'Las Palmas',
  'Atletico de Madrid',
  '1',
  '1',
  '12/09/1970',
  '21942000.0']]

## Tercera pregunta: Calcular el número total de goles que ha marcado el Real Sporting de Gijón.

In [179]:
# calculo local y visitante y los sumo
# Para cada uno, filtro el equipo, selecciono el valor de sus goles (núm. entero) y sumo
local = (parse_data.filter(lambda linea: linea[3] == "Sporting de Gijon")
                .map(lambda linea: int(linea[5])).sum())

visitante = (parse_data.filter(lambda linea: linea[4] == "Sporting de Gijon")
                .map(lambda linea: int(linea[6])).sum())

Total_goles = local + visitante

Total_goles

print("El Sporting de Gijón ha marcado {} goles en total.".format(Total_goles))
      

El Sporting de Gijón ha marcado 2042 goles en total.


## Cuarta pregunta: ¿ En que temporada se marcaron más goles?

In [176]:
# Tomo la temporada como clave, la suma de los goles locales y visitantes como valor, la funcción de suma accum en el reducebyKey y selecciono el mayor.
temp_goles = (parse_data.map(lambda linea: (linea[1], int(linea[5]) + int(linea[6])))
                       .reduceByKey(lambda a,b: a+b)
                        .takeOrdered(1, lambda x: -x[1]))

print("La temporada en la que se marcaron más goles fue la {}, en la que se marcaron {} goles en total.".format(temp_goles[0][0],temp_goles[0][1]))

La temporada en la que se marcaron más goles fue la 2012-13, en la que se marcaron 2294 goles en total.


## Quinta pregunta: ¿Cúal es el equipo y tiene el record de más goles como local? ¿ y cómo visitante?

In [177]:
# Tomo el equipo local de cada partido junto con sus respectivos goles
# aplico la función de suma
# seleeciono el primero

local_goles = parse_data.map(lambda linea:(linea[3],int(linea[5]))).reduceByKey(lambda a,b: a+b).takeOrdered(1,lambda x: -x[1])

print("El equipo más goleador como local es el {}, con {} goles en total.".format(local_goles[0][0],local_goles[0][1]))

El equipo más goleador como local es el Real Madrid, con 2054 goles en total.


In [178]:
# Tomo el equipo visitante de cada partido junto con sus respectivos goles
# aplico la función de suma
# seleeciono el primero
visit_goles = parse_data.map(lambda linea:(linea[4],int(linea[6]))).reduceByKey(lambda a,b: a+b).takeOrdered(1,lambda x: -x[1])

print("El equipo más goleador como visitante es el {}, con {} goles en total.".format(visit_goles[0][0],visit_goles[0][1]))

El equipo más goleador como visitante es el Barcelona, con 1296 goles en total.


## Sexta pregunta: ¿Cúales son las 3 décadas en las que más goles se metieron?

In [110]:
decadas = parse_data.map(lambda linea: (linea[7][-2],int(linea[5]) + int(linea[6]))).reduceByKey(lambda a,b: a+b).takeOrdered(3,lambda x: -x[1])

print("La década mas goleadora son los {}, con {} goles, en segundo lugar  los {} ({} goles) y en tercero los {} ({} goles).".format(decadas[0][0]*2,decadas[0][1],int(decadas[1][0])*10,decadas[1][1],int(decadas[2][0])*10,decadas[2][1]))

La década mas goleadora son los 00, con 20526 goles, en segundo lugar  los 90 (19321 goles) y en tercero los 80 (17336 goles).


##  Séptima pregunta: ¿Cúal es la media de victorias de los equipos que han estado menos de 10 temporadas en primera división?

In [ ]:
# no hay ningún campo que indique la división del partido, están todos los partidos de primera y de segunda sin distinción.
# Sabemos que At de Bilbao, Barcelona y Madrid, siempre han jugado en primera.
# Si ha estado al menos 10 temporadas en primera ha recibido al At. de Bilbao en al menos 10 ocasiones.

In [249]:
# Todos los equipos con sus respectivas temporadas en futbol profesional

Total_equipos = (parse_data.map(lambda linea: (linea[3], linea[1]))
     .groupByKey()
     .mapValues(set)
     .map(lambda x: (x[0], len(x[1]))))

# Equipos que han estado 10 o más temporadas en primera

Equipos_clasicos = (parse_data.filter(lambda linea: linea[4] == 'Athletic Club')
     .map(lambda linea: (linea[3], linea[1]))
     .groupByKey()
     .mapValues(len)
     .filter(lambda linea: linea[1] >= 10))

# equipos que no han estado 10 temporadas en primera
equipos_peques = (Total_equipos.leftOuterJoin(Equipos_clasicos)
                 .map(lambda linea: (linea[0],linea[1][1]))
                 .filter(lambda linea: (linea[1] == None)))

# Equipos que no han estado 10 o más temporadas en primera


In [251]:
# Partidos totales (calculo local y multiplico por dos, ya que siempre coinciden los partidos en casa y fuera en cada temporada)
partidos_total = (parse_data.map(lambda linea: (linea[3],1))
                   .reduceByKey(lambda a,b: a + b)
                   .map(lambda linea: (linea[0],int(linea[1])*2)))

# victorias como local
victorias_local = (parse_data.filter(lambda linea: int(linea[5]>linea[6]))
                   .map(lambda linea: (linea[3],1))
                   .reduceByKey(lambda a,b: a + b))

# victorias como visitante
victorias_visit = (parse_data.filter(lambda linea: int(linea[6]>linea[5]))
                   .map(lambda linea: (linea[4],1))
                   .reduceByKey(lambda a,b: a + b))

# Union de total de victorias por equipo
victorias_totales = victorias_local.union(victorias_visit).reduceByKey(lambda x,y : x+y)




In [253]:
# Uno los equipos que no han estado 10 temporadas en primera, con sus partidos totales y victoras totales.
# Calculo la media dividiendo el número de victorias totales entre el número total de partidos de cada uno.
media_victorias = (equipos_peques.join(partidos_total)
                   .join(victorias_totales)
                   .map(lambda linea: (linea[0],round(linea[1][1] / linea[1][0][1],4))))

# muestro los 10 que mejor media total tienen
media_victorias.takeOrdered(10,lambda x: -x[1])

[('Alcorcon', 0.4175),
 ('Athletic Club', 0.3983),
 ('Merida', 0.3814),
 ('Castilla', 0.3741),
 ('Toledo', 0.3633),
 ('Figueres', 0.3603),
 ('Elche', 0.3584),
 ('Girona', 0.3576),
 ('Sant Andreu', 0.3571),
 ('Villarreal B', 0.3571)]

##  Octava pregunta: ¿Cúal es la media de goles como visitante por partido?

In [112]:
visit_goles_media = parse_data.map(lambda linea: int(linea[6])).stats()
# alternativamente
visit_goles_media = parse_data.map(lambda linea: int(linea[6])).mean()

print("La media de goles de los equipos visitantes es de {} goles, por cada partido.".format(round(visit_goles_media,4)))

La media de goles de los equipos visitantes es de 0.928 goles, por cada partido
